In [4]:
import numpy as np
from pathlib import Path
import duckdb
import pandas as pd

from categoria_cid import CategoriaCID10
from tools import logger_basic_conf, drop_cols_by_null_percent

logger = logger_basic_conf()

In [5]:
DATA_DIR = "./data/"
data_path = Path(DATA_DIR)

In [6]:
CID_10_SUBCAT_FILE = "CID-10-SUBCATEGORIAS.CSV"
CID_10_CAT_FILE = "CID-10-CATEGORIAS.CSV"
CID_10_GRUPOS_FILE = "CID-10-GRUPOS.CSV"
CID_10_CAPITULOS_FILE = "CID-10-CAPITULOS.CSV"

df_cid_subcat = pd.read_csv(data_path / CID_10_SUBCAT_FILE, encoding="latin-1", sep=";")
df_cid_subcat["CAT"] = df_cid_subcat["SUBCAT"].str[:3].astype(str)

df_cid_cat = pd.read_csv(data_path / CID_10_CAT_FILE, encoding="latin-1", sep=";")
df_cid_cat["CAT"] = df_cid_cat["CAT"].astype(str)

df_cid = pd.merge(
    df_cid_subcat, df_cid_cat, on="CAT", how="inner", suffixes=("_SUBCAT", "_CAT")
)

df_cid_grupos = pd.read_csv(data_path / CID_10_GRUPOS_FILE, encoding="latin-1", sep=";")
df_cid_capitulos = pd.read_csv(
    data_path / CID_10_CAPITULOS_FILE, encoding="latin-1", sep=";"
)

df_cid["CAT_OBJ"] = df_cid["CAT"].apply(CategoriaCID10)

df_cid_grupos["CATINIC_OBJ"] = df_cid_grupos["CATINIC"].apply(CategoriaCID10)
df_cid_grupos["CATFIM_OBJ"] = df_cid_grupos["CATFIM"].apply(CategoriaCID10)

df_cid_capitulos["CATINIC_OBJ"] = df_cid_capitulos["CATINIC"].apply(CategoriaCID10)
df_cid_capitulos["CATFIM_OBJ"] = df_cid_capitulos["CATFIM"].apply(CategoriaCID10)


def find_interval(cat_obj, df_interval, start_col, end_col, label_col):
    for _, row in df_interval.iterrows():
        if row[start_col] <= cat_obj <= row[end_col]:
            return row[label_col]
    return None


df_cid["GRUPO"] = df_cid["CAT_OBJ"].apply(
    lambda x: find_interval(x, df_cid_grupos, "CATINIC_OBJ", "CATFIM_OBJ", "DESCRICAO")
)
df_cid["CAPITULO"] = df_cid["CAT_OBJ"].apply(
    lambda x: find_interval(
        x, df_cid_capitulos, "CATINIC_OBJ", "CATFIM_OBJ", "DESCRICAO"
    )
)

In [7]:
df_cid = drop_cols_by_null_percent(df_cid, null_limit=1, logger=logger)
df_cid = df_cid.drop(columns="CAT_OBJ")

df_cid

2025-10-27 18:20:06 - INFO - SUBCAT 0.00% nulos < limite 100.00%: APROVADO
2025-10-27 18:20:06 - INFO - CLASSIF_SUBCAT 96.96% nulos < limite 100.00%: APROVADO
2025-10-27 18:20:06 - INFO - RESTRSEXO 92.98% nulos < limite 100.00%: APROVADO
2025-10-27 18:20:06 - INFO - CAUSAOBITO 89.63% nulos < limite 100.00%: APROVADO
2025-10-27 18:20:06 - INFO - DESCRICAO_SUBCAT 0.00% nulos < limite 100.00%: APROVADO
2025-10-27 18:20:06 - INFO - DESCRABREV_SUBCAT 0.00% nulos < limite 100.00%: APROVADO
2025-10-27 18:20:06 - INFO - REFER_SUBCAT 98.74% nulos < limite 100.00%: APROVADO
2025-10-27 18:20:06 - INFO - EXCLUIDOS_SUBCAT 99.87% nulos < limite 100.00%: APROVADO
2025-10-27 18:20:06 - WARNING - Unnamed: 8 100.00% nulos >= limite 100.00% REPROVADO
2025-10-27 18:20:06 - INFO - CAT 0.00% nulos < limite 100.00%: APROVADO
2025-10-27 18:20:06 - INFO - CLASSIF_CAT 97.55% nulos < limite 100.00%: APROVADO
2025-10-27 18:20:06 - INFO - DESCRICAO_CAT 0.00% nulos < limite 100.00%: APROVADO
2025-10-27 18:20:06 - I

,SUBCAT,CLASSIF_SUBCAT,RESTRSEXO,CAUSAOBITO,DESCRICAO_SUBCAT,DESCRABREV_SUBCAT,REFER_SUBCAT,EXCLUIDOS_SUBCAT,CAT,CLASSIF_CAT,DESCRICAO_CAT,DESCRABREV_CAT,REFER_CAT,GRUPO,CAPITULO
0,A000,NaN,NaN,NaN,"Cólera devida a Vibrio cholerae 01, biótipo ch...",A00.0 Colera dev Vibrio cholerae 01 biot cholerae,NaN,NaN,A00,NaN,Cólera,A00 Colera,NaN,Doenças infecciosas intestinais,Capítulo I - Algumas doenças infecciosas e par...
1,A001,NaN,NaN,NaN,"Cólera devida a Vibrio cholerae 01, biótipo El...",A00.1 Colera dev Vibrio cholerae 01 biot El Tor,NaN,NaN,A00,NaN,Cólera,A00 Colera,NaN,Doenças infecciosas intestinais,Capítulo I - Algumas doenças infecciosas e par...
2,A009,NaN,NaN,NaN,Cólera não especificada,A00.9 Colera NE,NaN,NaN,A00,NaN,Cólera,A00 Colera,NaN,Doenças infecciosas intestinais,Capítulo I - Algumas doenças infecciosas e par...
3,A010,NaN,NaN,NaN,Febre tifóide,A01.0 Febre tifoide,NaN,NaN,A01,NaN,Febres tifóide e paratifóide,A01 Febres tifoide e paratifoide,NaN,Doenças infecciosas intestinais,Capítulo I - Algumas doenças infecciosas e par...
4,A011,NaN,NaN,NaN,Febre paratifóide A,A01.1 Febre paratifoide A,NaN,NaN,A01,NaN,Febres tifóide e paratifóide,A01 Febres tifoide e paratifoide,NaN,Doenças infecciosas intestinais,Capítulo I - Algumas doenças infecciosas e par...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12446,U818,NaN,NaN,NaN,Agente resistente a outros antibióticos relaci...,U81.8 Agente resist outr antibiót relac vancom...,NaN,NaN,U81,NaN,Agente resistente à vancomicina e antibióticos...,U81 Agente resist vancomicina e antibiót relac,NaN,Agentes bacterianos resistentes a antibióticos,Capítulo XXII - Códigos para propósitos especiais
12447,U88,NaN,NaN,NaN,Agente resistente a múltiplos antibióticos,U88 Agente resistente a múltiplos antibióticos,NaN,NaN,U88,NaN,Agente resistente a múltiplos antibióticos,U88 Agente resistente a múltiplos antibióticos,NaN,Agentes bacterianos resistentes a antibióticos,Capítulo XXII - Códigos para propósitos especiais
12448,U898,NaN,NaN,NaN,Agente resistente a outro antibiótico especifi...,U89.8 Agente resist outro antibiótico espec único,NaN,NaN,U89,NaN,Agente resistente a outros antibióticos e a an...,U89 Agente resist outr antibiót e antibiót NE,NaN,Agentes bacterianos resistentes a antibióticos,Capítulo XXII - Códigos para propósitos especiais
12449,U899,NaN,NaN,NaN,Agente resistente a antibiótico não especificado,U89.9 Agente resistente a antibiótico NE,NaN,NaN,U89,NaN,Agente resistente a outros antibióticos e a an...,U89 Agente resist outr antibiót e antibiót NE,NaN,Agentes bacterianos resistentes a antibióticos,Capítulo XXII - Códigos para propósitos especiais


In [8]:
cid_dict_subcategoria = {
    code: {
        "cid_subcategoria": cid_subcategoria,
        "cid_categoria": cid_categoria,
        "cid_grupo": cid_grupo,
        "cid_capitulo": cid_capitulo,
    }
    for code, cid_subcategoria, cid_categoria, cid_grupo, cid_capitulo in zip(
        df_cid["SUBCAT"],
        df_cid["DESCRICAO_SUBCAT"],
        df_cid["DESCRICAO_CAT"],
        df_cid["GRUPO"],
        df_cid["CAPITULO"],
    )
}

cid_dict_categoria = {
    code: {
        "cid_categoria": cid_categoria,
        "cid_grupo": cid_grupo,
        "cid_capitulo": cid_capitulo,
    }
    for code, cid_categoria, cid_grupo, cid_capitulo in set(
        zip(
            df_cid["CAT"],
            df_cid["DESCRICAO_CAT"],
            df_cid["GRUPO"],
            df_cid["CAPITULO"],
        )
    )
}

In [9]:
CBO_FILE_TRANSLATOR = "CBO94 - CBO2002 - Conversao.csv"

df_cbo_tradutor = pd.read_csv(
    data_path / CBO_FILE_TRANSLATOR, sep=";", encoding="latin-1"
)

display(df_cbo_tradutor)

cbo_tradutor_dict = {
    str(cbo94).replace("X", ""): str(cbo2002).zfill(6)
    for cbo94, cbo2002 in zip(df_cbo_tradutor["CBO94"], df_cbo_tradutor["CBO2002"])
}

cbo_tradutor_dict

,CBO94,CBO2002
0,X4010,20105
1,X4020,20205
2,X4030,21105
3,X4040,21210
4,X5010,10110
...,...,...
1969,98940,782145
1970,98945,783225
1971,98950,782820
1972,99130,516610


{'4010': '020105',
 '4020': '020205',
 '4030': '021105',
 '4040': '021210',
 '5010': '010110',
 '5020': '010210',
 '5030': '010310',
 '5040': '010310',
 '6010': '010115',
 '6020': '010215',
 '6030': '010215',
 '6040': '010315',
 '7010': '020105',
 '7020': '020205',
 '7030': '021110',
 '7040': '021205',
 '8010': '020105',
 '8020': '020205',
 '8030': '021110',
 '8040': '021205',
 '01105': '213210',
 '01110': '213205',
 '01125': '213205',
 '01145': '213210',
 '01150': '213205',
 '01155': '213205',
 '01210': '213105',
 '01215': '213150',
 '01220': '213105',
 '01230': '213175',
 '01235': '213105',
 '01240': '213160',
 '01250': '213110',
 '01260': '213105',
 '01270': '213105',
 '01280': '213155',
 '01920': '213415',
 '01930': '213315',
 '01940': '213305',
 '01950': '203215',
 '02020': '222110',
 '02040': '222120',
 '02060': '222115',
 '02110': '214205',
 '02115': '214215',
 '02125': '214255',
 '02135': '214210',
 '02145': '214225',
 '02150': '214245',
 '02155': '214265',
 '02160': '214230',


In [10]:
input_path = data_path / "CBO2002 - Ocupacao.csv"

df_cbo = pd.read_csv(
    input_path, sep=";", encoding="latin-1", engine="python", on_bad_lines="warn"
)

display(df_cbo)

,CODIGO,TITULO
0,10105,Oficial general da aeronáutica
1,10110,Oficial general do exército
2,10115,Oficial general da marinha
3,10205,Oficial da aeronáutica
4,10210,Oficial do exército
...,...,...
2714,992205,Encarregado geral de operações de conservação ...
2715,992210,Encarregado de equipe de conservação de vias p...
2716,992215,Operador de ceifadeira na conservação de vias ...
2717,992220,Pedreiro de conservação de vias permanentes (e...


In [11]:
cbo_dict = {
    str(code): titulo for code, titulo in zip(df_cbo["CODIGO"], df_cbo["TITULO"])
}

for extra_code, titulo in [
    ("999991", "Estudante"),
    ("999992", "Dona de casa"),
    ("999993", "Aposentado"),
    ("999995", "Presidiario"),
]:
    cbo_dict[extra_code] = titulo

In [12]:
IBGE_FILE = "br_bd_diretorios_brasil_municipio.csv"

df_ibge = pd.read_csv(data_path / IBGE_FILE)

display(df_ibge)

,id_municipio,id_municipio_6,id_municipio_tse,id_municipio_rf,id_municipio_bcb,nome,capital_uf,id_comarca,id_regiao_saude,nome_regiao_saude,...,nome_mesorregiao,id_regiao_metropolitana,nome_regiao_metropolitana,ddd,id_uf,sigla_uf,nome_uf,nome_regiao,amazonia_legal,centroide
0,5101837,510183,NaN,NaN,NaN,Boa Esperança do Norte,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,51,MT,Mato Grosso,Centro-Oeste,1,POINT(-54.9417429754238 -13.463197935201)
1,1100809,110080,477.0,681.0,46961.0,Candeias do Jamari,0.0,1100205.0,11004.0,Madeira-Mamoré,...,Madeira-Guaporé,101,Região Metropolitana de Porto Velho,69.0,11,RO,Rondônia,Norte,1,POINT(-63.3254198532114 -8.88702392955617)
2,1100338,110033,434.0,47.0,44516.0,Nova Mamoré,0.0,1100106.0,11004.0,Madeira-Mamoré,...,Madeira-Guaporé,NaN,NaN,69.0,11,RO,Rondônia,Norte,1,POINT(-64.6295025002404 -10.3822863210755)
3,1100205,110020,35.0,3.0,30719.0,Porto Velho,1.0,1100205.0,11004.0,Madeira-Mamoré,...,Madeira-Guaporé,101,Região Metropolitana de Porto Velho,69.0,11,RO,Rondônia,Norte,1,POINT(-64.3041357999869 -9.15394033687075)
4,1101104,110110,493.0,683.0,46851.0,Itapuã do Oeste,0.0,1100205.0,11004.0,Madeira-Mamoré,...,Madeira-Guaporé,NaN,NaN,69.0,11,RO,Rondônia,Norte,1,POINT(-63.044960463046 -9.17019402601564)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5566,5209150,520915,92266.0,9799.0,43830.0,Gouvelândia,0.0,5218300.0,52017.0,Sul,...,Sul Goiano,NaN,NaN,64.0,52,GO,Goiás,Centro-Oeste,0,POINT(-50.1557720034523 -18.509097862386)
5567,5220405,522040,96059.0,9605.0,36799.0,São Simão,0.0,5220405.0,52015.0,Sudoeste I,...,Sul Goiano,NaN,NaN,64.0,52,GO,Goiás,Centro-Oeste,0,POINT(-50.5908964743097 -19.0104603156912)
5568,5216304,521630,94552.0,9455.0,24550.0,Paranaiguara,0.0,5216304.0,52015.0,Sudoeste I,...,Sul Goiano,NaN,NaN,64.0,52,GO,Goiás,Centro-Oeste,0,POINT(-50.6237723254387 -18.8084672995792)
5569,5218508,521850,95630.0,9563.0,4611.0,Quirinópolis,0.0,5218508.0,52015.0,Sudoeste I,...,Sul Goiano,NaN,NaN,64.0,52,GO,Goiás,Centro-Oeste,0,POINT(-50.5293403684736 -18.4373825773833)


In [13]:
ibge_dict = {
    str(code): {
        "municipio": municipio,
        "regiao_imediata": regiao_imediata,
        "regiao_intermediaria": regiao_intermediaria,
        "uf": uf,
        "regiao": regiao,
    }
    for code, municipio, regiao_imediata, regiao_intermediaria, uf, regiao in zip(
        df_ibge["id_municipio"],
        df_ibge["nome"],
        df_ibge["nome_regiao_imediata"],
        df_ibge["nome_regiao_intermediaria"],
        df_ibge["sigla_uf"],
        df_ibge["nome_regiao"],
    )
}

ibge_6_dict = {
    str(code): {
        "municipio": municipio,
        "regiao_imediata": regiao_imediata,
        "regiao_intermediaria": regiao_intermediaria,
        "uf": uf,
        "regiao": regiao,
    }
    for code, municipio, regiao_imediata, regiao_intermediaria, uf, regiao in zip(
        df_ibge["id_municipio_6"],
        df_ibge["nome"],
        df_ibge["nome_regiao_imediata"],
        df_ibge["nome_regiao_intermediaria"],
        df_ibge["sigla_uf"],
        df_ibge["nome_regiao"],
    )
}

In [14]:
df_ibge[df_ibge["nome"] == "Curitiba"]

,id_municipio,id_municipio_6,id_municipio_tse,id_municipio_rf,id_municipio_bcb,nome,capital_uf,id_comarca,id_regiao_saude,nome_regiao_saude,...,nome_mesorregiao,id_regiao_metropolitana,nome_regiao_metropolitana,ddd,id_uf,sigla_uf,nome_uf,nome_regiao,amazonia_legal,centroide
4285,4106902,410690,75353.0,7535.0,13853.0,Curitiba,1.0,4106902.0,41002.0,2ª RS Metropolitana,...,Metropolitana de Curitiba,5501,Região Metropolitana de Curitiba,41.0,41,PR,Paraná,Sul,0,POINT(-49.288257976141 -25.4778625693795)


In [15]:
DUCKDB_FILE = "sim.duckdb"

# Filtro de município -> Curitiba, Local de ocorrência -> via pública
con = duckdb.connect(database=data_path / DUCKDB_FILE, read_only=True)
df_sim = con.execute(
    "SELECT * FROM sim WHERE (CODMUNOCOR = '4106902' or CODMUNOCOR = '410690') AND LOCOCOR = '4'"
).fetchdf()
con.close()

df_sim = drop_cols_by_null_percent(df_sim, 0.5, "LINHA", logger)

df_sim

2025-10-27 18:20:27 - INFO - id 0.00% nulos < limite 50.00%: APROVADO
2025-10-27 18:20:27 - WARNING - CRSOCOR 100.00% nulos >= limite 50.00% REPROVADO
2025-10-27 18:20:27 - WARNING - OCUPACAO 100.00% nulos >= limite 50.00% REPROVADO
2025-10-27 18:20:27 - INFO - IDADE 0.51% nulos < limite 50.00%: APROVADO
2025-10-27 18:20:27 - WARNING - PESONASC 100.00% nulos >= limite 50.00% REPROVADO
2025-10-27 18:20:27 - WARNING - AREARES 100.00% nulos >= limite 50.00% REPROVADO
2025-10-27 18:20:28 - WARNING - OCUPPAI 100.00% nulos >= limite 50.00% REPROVADO
2025-10-27 18:20:28 - WARNING - FONTINFO 100.00% nulos >= limite 50.00% REPROVADO
2025-10-27 18:20:28 - WARNING - DATAREG 100.00% nulos >= limite 50.00% REPROVADO
2025-10-27 18:20:28 - WARNING - INSTRUCAO 100.00% nulos >= limite 50.00% REPROVADO
2025-10-27 18:20:28 - INFO - CAUSABAS 0.00% nulos < limite 50.00%: APROVADO
2025-10-27 18:20:28 - WARNING - FILHVIVOS 100.00% nulos >= limite 50.00% REPROVADO
2025-10-27 18:20:28 - WARNING - CODIGO 100.00

,id,IDADE,CAUSABAS,TIPOBITO,NECROPSIA,CIRURGIA,SEXO,LOCOCOR,EXAME,NATURAL,...,LINHAD,DTINVESTIG,DTRECEBIM,FONTEINV,DTCADASTRO,TPPOS,CAUSABAS_O,DTATESTADO,HORAOBITO,ORIGEM
0,13706772,427,V892,2,1,2,1,4,2,852,...,None,None,None,None,None,None,None,None,None,None
1,14043004,451,R98,2,None,None,1,4,None,800,...,None,None,None,None,None,None,None,None,None,None
2,14047392,455,I64,2,2,2,2,4,1,841,...,None,None,None,None,None,None,None,None,None,None
3,14047656,422,X954,2,1,2,1,4,2,833,...,None,None,None,None,None,None,None,None,None,None
4,14047660,429,V892,2,1,2,1,4,2,800,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12184,47748975,446,R99,2,1,,1,4,,841,...,,,23092024,,20092024,N,R99,15092024,1730,1
12185,47748978,429,V051,2,1,,1,4,,841,...,,,03102024,,03102024,N,V051,15092024,0300,1
12186,47748992,435,Y244,2,1,,1,4,,841,...,,,23092024,,20092024,N,Y244,18092024,0900,1
12187,47749654,470,I802,2,2,,1,4,,841,...,,24102024,29102024,8,27092024,S,I802,25092024,1730,1


In [16]:
df_sim.head()

,id,IDADE,CAUSABAS,TIPOBITO,NECROPSIA,CIRURGIA,SEXO,LOCOCOR,EXAME,NATURAL,...,LINHAD,DTINVESTIG,DTRECEBIM,FONTEINV,DTCADASTRO,TPPOS,CAUSABAS_O,DTATESTADO,HORAOBITO,ORIGEM
0,13706772,427,V892,2,1,2,1,4,2,852,...,None,None,None,None,None,None,None,None,None,None
1,14043004,451,R98,2,None,None,1,4,None,800,...,None,None,None,None,None,None,None,None,None,None
2,14047392,455,I64,2,2,2,2,4,1,841,...,None,None,None,None,None,None,None,None,None,None
3,14047656,422,X954,2,1,2,1,4,2,833,...,None,None,None,None,None,None,None,None,None,None
4,14047660,429,V892,2,1,2,1,4,2,800,...,None,None,None,None,None,None,None,None,None,None


<h2>Dicionário de Dados</h2>
<table border="1" cellspacing="0" cellpadding="5">
  <thead>
    <tr>
      <th>Nome Original</th>
      <th>Nome Normalizado</th>
      <th>Categoria</th>
      <th>Observações</th>
    </tr>
  </thead>
  <tbody>
    <tr><td>CONTADOR</td><td>contador</td><td>Metadado</td><td></td></tr>
    <tr><td>ORIGEM</td><td>origem</td><td>Metadado</td><td>1 – Oracle; 2 – Banco estadual via FTP; 3 – Banco SEADE; 9 – Ignorado</td></tr>
    <tr><td>DTNASC</td><td>data_nascimento</td><td>Falecido</td><td>ddmmaaaa</td></tr>
    <tr><td>IDADE</td><td>idade</td><td>Falecido</td><td>O primeiro dígito indica a unidade da idade (1=minuto, 2=hora, 3=mês, 4=ano, 5&gt;100 anos); o segundo dígito indica a quantidade de unidades</td></tr>
    <tr><td>NATURAL</td><td>naturalidade</td><td>Falecido</td><td></td></tr>
    <tr><td>CODMUNRES</td><td>municipio_residencia</td><td>Falecido</td><td>Código IBGE</td></tr>
    <tr><td>SEXO</td><td>sexo</td><td>Falecido</td><td>1 – masculino; 2 – feminino</td></tr>
    <tr><td>RACACOR</td><td>raca_cor</td><td>Falecido</td><td>1 – Branca; 2 – Preta; 3 – Amarela; 4 – Parda; 5 – Indígena</td></tr>
    <tr><td>ESTCIV</td><td>situacao_conjugal</td><td>Falecido</td><td>1 – Solteiro; 2 – Casado; 3 – Viúvo; 4 – Separado/divorciado; 5 – União estável; 9 – Ignorado</td></tr>
    <tr><td>ESC</td><td>escolaridade_em_anos</td><td>Falecido</td><td>1 – Nenhuma; 2 – 1 a 3 anos; 3 – 4 a 7 anos; 4 – 8 a 11 anos; 5 – 12 anos ou mais; 9 – Ignorado</td></tr>
    <tr><td>OCUP</td><td>ocupacao</td><td>Falecido</td><td>CBO 2002</td></tr>
    <tr><td>DTOBITO</td><td>data_obito</td><td>Óbito</td><td>ddmmaaaa</td></tr>
    <tr><td>HORAOBITO</td><td>hora_obito</td><td>Óbito</td><td>hhmm</td></tr>
    <tr><td>LOCOCOR</td><td>local_obito</td><td>Óbito</td><td>1 – hospital; 2 – outros estabelecimentos; 3 – domicílio; 4 – via pública; 5 – outros; 6 – aldeia indígena; 9 – ignorado</td></tr>
    <tr><td>CODMUNOCOR</td><td>municipio_obito</td><td>Óbito</td><td>Código IBGE</td></tr>
    <tr><td>CODEESTAB</td><td>estabelecimento</td><td>Óbito</td><td>CNES</td></tr>
    <tr><td>CAUSABAS</td><td>causa_obito</td><td>Óbito</td><td>CID-10</td></tr>
    <tr><td>CAUSABAS_O</td><td>causa_obito_origem</td><td>Óbito</td><td>CID-10</td></tr>
    <tr><td>LINHAA</td><td>causas_obito_1a</td><td>Óbito</td><td>CID-10</td></tr>
    <tr><td>LINHAB</td><td>causas_obito_1b</td><td>Óbito</td><td>CID-10</td></tr>
    <tr><td>LINHAC</td><td>causas_obito_1c</td><td>Óbito</td><td>CID-10</td></tr>
    <tr><td>LINHAD</td><td>causas_obito_1d</td><td>Óbito</td><td>CID-10</td></tr>
    <tr><td>LINHAII</td><td>causas_obito_2</td><td>Óbito</td><td>CID-10</td></tr>
    <tr><td>ASSISTMED</td><td>recebeu_assistencia_medica</td><td>Óbito</td><td>1 – sim; 2 – não; 9 – ignorado</td></tr>
    <tr><td>EXAME</td><td>realizacao_de_exame</td><td>Óbito</td><td>1 – sim; 2 – não; 9 – ignorado</td></tr>
    <tr><td>CIRURGIA</td><td>realizacao_de_cirurgia</td><td>Óbito</td><td>1 – sim; 2 – não; 9 – ignorado</td></tr>
    <tr><td>TIPOBITO</td><td>obito_fetal</td><td>Óbito</td><td>1 – Fetal; 2 – Não fetal</td></tr>
    <tr><td>NECROPSIA</td><td>necropsia</td><td>Post-mortem</td><td>1 – sim; 2 – não; 9 – ignorado</td></tr>
    <tr><td>TPPOS</td><td>investigado</td><td>Post-mortem</td><td>S – sim; N – não</td></tr>
    <tr><td>DTATESTADO</td><td>data_atestado</td><td>Post-mortem</td><td>ddmmaaaa</td></tr>
    <tr><td>DTCADASTRO</td><td>data_cadastro</td><td>Post-mortem</td><td>ddmmaaaa</td></tr>
    <tr><td>DTRECEBIM</td><td>data_recebimento</td><td>Post-mortem</td><td>ddmmaaaa</td></tr>
    <tr><td>ATESTANTE</td><td>atestante</td><td>Post-mortem</td><td>1 – Assistente; 2 – Substituto; 3 – IML; 4 – SVO; 5 – Outro</td></tr>
  </tbody>
</table>


In [17]:
# traduzir colunas
df_col_dict = {
    # metadado
    "CONTADOR": "contador",
    "ORIGEM": "origem",
    # falecido
    "DTNASC": "data_nascimento",
    "IDADE": "idade",
    "NATURAL": "naturalidade",
    "CODMUNRES": "municipio_residencia",
    "SEXO": "sexo",
    "RACACOR": "raca_cor",
    "ESTCIV": "situacao_conjugal",
    "ESC": "escolaridade_em_anos",
    "OCUP": "ocupacao",
    # falecimento
    "DTOBITO": "data_obito",
    "HORAOBITO": "hora_obito",
    "LOCOCOR": "local_obito",
    "CODMUNOCOR": "municipio_obito",
    "CAUSABAS": "causa_obito",
    "CAUSABAS_O": "causa_obito_origem",
    "LINHAA": "causas_obito_1a",
    "LINHAB": "causas_obito_1b",
    "LINHAC": "causas_obito_1c",
    "LINHAD": "causas_obito_1d",
    "LINHAII": "causas_obito_2",
    "ASSISTMED": "recebeu_assistencia_medica",
    "EXAME": "realizacao_de_exame",
    "CIRURGIA": "realizacao_de_cirurgia",
    "TIPOBITO": "obito_fetal",
    # post-mortem
    "NECROPSIA": "necropsia",
    "TPPOS": "investigado",
    "DTATESTADO": "data_atestado",
    "DTCADASTRO": "data_cadastro",  # ddmmaaaa
    "DTRECEBIM": "data_recebimento",  # ddmmaaaa
    "ATESTANTE": "atestante",
}

df_sim_mapped = df_sim.rename(columns=df_col_dict)
df_sim_mapped = df_sim_mapped[list(df_col_dict.values())]

In [18]:
# 'contador' indica o número do óbito no ano; como o dataset já foi filtrado, não acrescenta informação relevante.
df_sim_mapped = df_sim_mapped.drop(columns="contador")

# A coluna 'naturalidade' utiliza um código municipal distinto das demais e não pôde ser decodificada.
df_sim_mapped = df_sim_mapped.drop(columns=["naturalidade"])

# As colunas 'causas_obito_*' são listas de causas, e por simplificação foram removidas.
for col in [c for c in df_sim_mapped.columns if c.startswith("causas_obito_")]:
    df_sim_mapped = df_sim_mapped.drop(columns=col)

In [19]:
for col in [c for c in df_sim_mapped.columns if c.startswith("data_")]:
    df_sim_mapped[col] = pd.to_datetime(
        df_sim_mapped[col], format="%d%m%Y", errors="coerce"
    )

for col in [c for c in df_sim_mapped.columns if c.startswith("hora_")]:
    df_sim_mapped[col] = pd.to_datetime(
        df_sim_mapped[col], format="%H%M", errors="coerce"
    ).dt.time

In [20]:
for col in [c for c in df_sim_mapped.columns if c.startswith("municipio_")]:
    mapped = (
        df_sim_mapped[col]
        .astype(str)
        .map(ibge_dict)
        .fillna(df_sim_mapped[col].map(ibge_6_dict))
    )

    df_exploded = pd.json_normalize(mapped).add_prefix(f"{col}_")
    df_sim_mapped = pd.concat([df_sim_mapped, df_exploded], axis=1)

In [21]:
for col in [c for c in df_sim_mapped.columns if c.startswith("causa_obito")]:
    mapped = [
        cid_dict_categoria.get(cid[:3], cid)
        if cid and (cid.endswith("X") or len(cid) == 3)
        else cid_dict_subcategoria.get(cid, cid)
        for cid in df_sim_mapped[col]
    ]

    df_exploded = pd.json_normalize(mapped).add_prefix(f"{col}_")
    df_sim_mapped = pd.concat([df_sim_mapped, df_exploded], axis=1)

In [22]:
df_sim_mapped["ocupacao"] = (
    df_sim_mapped["ocupacao"]
    .str.zfill(5)
    .map(cbo_tradutor_dict)
    .fillna(df_sim_mapped["ocupacao"])
)


mapped = (
    df_sim_mapped["ocupacao"]
    .str.zfill(6)
    .map(cbo_dict)
    .fillna(df_sim_mapped["ocupacao"])
)

df_exploded = pd.json_normalize(mapped).add_prefix("ocupacao_")
df_sim_mapped = pd.concat([df_sim_mapped, df_exploded], axis=1)

In [23]:
origem_dict = {"1": "oracle", "2": "banco_estadual", "3": "seade", "9": "ignorado"}

df_sim_mapped["origem"] = (
    df_sim_mapped["origem"].map(origem_dict).fillna(df_sim_mapped["origem"])
)

In [24]:
unidade_idade_dict = {
    "1": "minuto",
    "2": "hora",
    "3": "mes",
    "4": "ano",
    "5": "maior_100_anos",
}

idade_str = df_sim_mapped["idade"].astype(str).replace("nan", np.nan)
df_sim_mapped["unidade_idade"] = idade_str.str[0].map(unidade_idade_dict)
idade_val = idade_str.str[1:]
df_sim_mapped["idade"] = pd.to_numeric(idade_val, errors="coerce")

In [25]:
sexo_dict = {
    "1": "masculino",
    "2": "feminino",
}

df_sim_mapped["sexo"] = (
    df_sim_mapped["sexo"]
    .map(sexo_dict)
    .fillna(df_sim_mapped["sexo"])
    .fillna(df_sim_mapped["sexo"])
)

In [26]:
raca_cor_dict = {
    "1": "branca",
    "2": "preta",
    "3": "amarela",
    "4": "parda",
    "5": "indigena",
}

df_sim_mapped["raca_cor"] = (
    df_sim_mapped["raca_cor"]
    .map(raca_cor_dict, na_action="ignore")
    .fillna(df_sim_mapped["raca_cor"])
)

In [27]:
estciv_dict = {
    "1": "solteiro",
    "2": "casado",
    "3": "viuvo",
    "4": "separado_judicialmente_divorciado",
    "5": "uniao_estavel",
    "9": "ignorado",
}

df_sim_mapped["situacao_conjugal"] = (
    df_sim_mapped["situacao_conjugal"]
    .map(estciv_dict, na_action="ignore")
    .fillna(df_sim_mapped["situacao_conjugal"])
)

In [28]:
esc_dict = {
    "1": "nenhuma",
    "2": "de_1_a_3_anos",
    "3": "de_4_a_7_anos",
    "4": "de_8_a_11_anos",
    "5": "de_12_anos_e_mais",
    "9": "ignorado",
}

df_sim_mapped["escolaridade_em_anos"] = (
    df_sim_mapped["escolaridade_em_anos"]
    .map(esc_dict, na_action="ignore")
    .fillna(df_sim_mapped["escolaridade_em_anos"])
)

In [29]:
local_obito_dict = {
    "1": "hospital",
    "2": "outros_estabelecimentos_de_saude",
    "3": "domicilio",
    "4": "via_publica",
    "5": "outros",
    "6": "aldeia_indigena",
    "9": "ignorado",
}

df_sim_mapped["local_obito"] = (
    df_sim_mapped["local_obito"]
    .map(local_obito_dict, na_action="ignore")
    .fillna(df_sim_mapped["local_obito"])
)

# como está filtrada na busca, todas as ocorrências são "via pública"
df_sim_mapped.drop(columns=["local_obito"], inplace=True)

In [30]:
sim_nao_ignorado_dict = {
    "1": "sim",
    "2": "nao",
    "9": "ignorado",
}

for col in [
    "recebeu_assistencia_medica",
    "realizacao_de_exame",
    "realizacao_de_cirurgia",
    "necropsia",
]:
    df_sim_mapped[col] = (
        df_sim_mapped[col]
        .map(sim_nao_ignorado_dict, na_action="ignore")
        .fillna(df_sim_mapped[col])
    )

s_n_dict = {"S": "sim", "N": "nao"}

df_sim_mapped["investigado"] = (
    df_sim_mapped["investigado"]
    .map(s_n_dict, na_action="ignore")
    .fillna(df_sim_mapped["investigado"])
)

In [31]:
obito_fetal_dict = {
    "1": "fetal",
    "2": "nao_fetal",
}

df_sim_mapped["obito_fetal"] = (
    df_sim_mapped["obito_fetal"]
    .map(obito_fetal_dict, na_action="ignore")
    .fillna(df_sim_mapped["obito_fetal"])
)

In [32]:
atestante_dict = {
    "1": "assistente",
    "2": "substituto",
    "3": "iml",
    "4": "svo",
    "5": "outro",
}

df_sim_mapped["atestante"] = (
    df_sim_mapped["atestante"]
    .map(atestante_dict, na_action="ignore")
    .fillna(df_sim_mapped["atestante"])
)

In [33]:
display(df_sim_mapped.obito_fetal.value_counts())

# 'obito_fetal' contém apenas uma categoria após o filtro por município.
df_sim_mapped = df_sim_mapped.drop(columns=["obito_fetal"])


obito_fetal
nao_fetal    12189
Name: count, dtype: int64

In [34]:
df_sim_mapped = df_sim_mapped.replace("", None)

In [35]:
dataset_path = Path("./dataset/")

df_sim_mapped.to_csv(dataset_path / "curitiba_via_publica.csv", index=False)